## Accessing the project directory on my Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/drive', force_remount=False)

Mounted at /drive


In [ ]:
project_dir = "/drive/My Drive/RNN_seq2seq"

# change working directory to project_dir
os.chdir(project_dir)

## Dependencies

In [ ]:
import torch
import torch.nn as nn

from os import makedirs
from os.path import join

import pandas as pd
import matplotlib.pyplot as plt
from string import ascii_lowercase

from scripts.data import get_input_spec_red_pair
from scripts.visualization import plot_performances_per_seq_len
from scripts.utils import read_data, read_json, read_datasets, save_dict_as_json
from scripts.dataloader import get_text_encoder_decoder, customize_dataloader_func
from scripts.pytorch_utils import get_model, customize_predictor, metrics, evaluate, get_results

## Log time and random seeds

In [ ]:
from datetime import datetime

now = datetime.now()
print("Time stamp:", now.strftime("%Y-%m-%d %H:%M:%S"))

Time stamp: 2023-06-07 03:33:41


## Data loading and processing

- I intended to make 35 copies, but actually I made a mistake here. In fact,`thirty_five_copies` produces 40 copies, instead of 35 copies.

In [ ]:
train, dev, test, gen = read_datasets("data/identity")


def thirty_five_copies(w):
    return w * 40


def modify_dataset(ds):

    out = []
    for t, _ in ds:
        out.append([t, thirty_five_copies(t)])

    return out

train = modify_dataset(train)
dev = modify_dataset(dev)
test = modify_dataset(test)
gen = modify_dataset(gen)

In [ ]:
len(train), len(dev), len(test), len(gen)

(10000, 10000, 50000, 100000)

In [ ]:
in_vocab = ascii_lowercase
out_vocab = ascii_lowercase
in_seq_encoder, in_seq_decoder = get_text_encoder_decoder(in_vocab)
out_seq_encoder, out_seq_decoder = get_text_encoder_decoder(out_vocab)

In [ ]:
dataloader_func = customize_dataloader_func(in_seq_encoder,
                                            out_seq_encoder,
                                            padding_idx=1,
                                            batch_size=1000,
                                            shuffle=False)

train_dl = dataloader_func(train)
dev_dl = dataloader_func(dev)
test_dl = dataloader_func(test, batch_size=5000)
gen_dl = dataloader_func(gen, batch_size=5000)

## Experiments

In [ ]:
def get_trained_model(task_name, rnn_type, attention, run_num):
    attention = "attn" if attention else "attn-less"
    saved_model_fp = f'experiments/{task_name}/{rnn_type}/{attention}/{run_num}/model.pt'
    ModelConfig = read_json(f'experiments/{task_name}/{rnn_type}/{attention}/{run_num}/ModelConfig.json')
    model = get_model(ModelConfig)
    model.load_state_dict(torch.load(saved_model_fp))
    return model

In [ ]:
task_name = "quadratic_copying"

main_results = []
task_folder = join("experiments", task_name + "attn_40_copies_RE_EVAL")
meta_col = ["RNN", "Attention", "lr", "Hidden size", "Embedding size", "Param #", "Max Epoch #"]
main_res_col = ["Run #", "RNN", "Attention", "Dataset", "Loss",
                "Full Sequence Accuracy", "First N-symbol Accuracy", "Overlap Rate"]


for rnn_type in ["SRNN", "GRU", "LSTM"]:
    for use_attn in [True]:
        for run_num in range(1, 4):

            print(f"RNN type: {rnn_type}; use attention: {use_attn}; run num: {run_num}\n")

            model = get_trained_model(task_name, rnn_type, use_attn, run_num)
            criterion = nn.CrossEntropyLoss()

            attn = "attn" if use_attn else "attn-less"
            folder = join(task_folder, rnn_type, attn, str(run_num))
            makedirs(folder, exist_ok=True)

            train_aggr, perfs_train = evaluate(model, train_dl, criterion,
                                               per_seq_len_performance=True)
            dev_aggr, perfs_dev = evaluate(model, dev_dl, criterion,
                                           per_seq_len_performance=True)
            test_aggr, perfs_test = evaluate(model, test_dl, criterion,
                                             per_seq_len_performance=True)
            gen_aggr, perfs_gen = evaluate(model, gen_dl, criterion,
                                             per_seq_len_performance=True)

            train_res = get_results(train_aggr, train_log=False)
            dev_res = get_results(dev_aggr, train_log=False)
            test_res = get_results(test_aggr, train_log=False)
            gen_res = get_results(gen_aggr, train_log=False)

            for res, ds in zip([train_res, dev_res, test_res, gen_res], ["Train", "Dev", "Test", "Gen"]):
                main_results.append([run_num, rnn_type, use_attn, ds] + res)

            save_dict_as_json(perfs_train, join(folder, "perfs_train.json"))
            save_dict_as_json(perfs_dev, join(folder, "perfs_dev.json"))
            save_dict_as_json(perfs_test, join(folder, "perfs_test.json"))
            save_dict_as_json(perfs_gen, join(folder, "perfs_gen.json"))
            plot_performances_per_seq_len(perfs_train, False, join(folder, "perfs_train.png"))
            plot_performances_per_seq_len(perfs_dev, False, join(folder, "perfs_dev.png"))
            plot_performances_per_seq_len(perfs_test, False, join(folder, "perfs_test.png"))
            plot_performances_per_seq_len(perfs_gen, False, join(folder, "perfs_gen.png"))

pd.DataFrame(main_results, columns=main_res_col).to_csv(join(task_folder, "main_results.csv"), index=False)

RNN type: SRNN; use attention: True; run num: 1

The model has 1,466,396 trainable parameters
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_train.json saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_dev.json saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_test.json saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_gen.json saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_train.png saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_dev.png saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_test.png saved!
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/1/perfs_gen.png saved!
RNN type: SRNN; use attention: True; run num: 2

The model has 1,466,396 trainable parameters
experiments/quadratic_copyingattn_35_copies_RE_EVAL/SRNN/attn/2/perfs_train.json saved!
experiments/quadratic_copyingattn_

## Automatically disconnect and delete the runtime

In [ ]:
from google.colab import runtime
runtime.unassign()